In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.13.1+cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 34.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 10.4 MB/s eta 0:00:00


In [ ]:
from ast import mod
from torch_geometric.sampler.neighbor_sampler import torch_geometric


import argparse
import os
import torch as th
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.nn import Linear
from sklearn.model_selection import train_test_split
import numpy as np
from torch_geometric.datasets import TUDataset
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader
from torch_geometric.explain import Explainer, GNNExplainer
import torch_geometric.nn as gnn


class GlobalMeanPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return gnn.global_mean_pool(x, batch)
################################################################################
class IdenticalPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return x

################################################################################
class Graph_Net(torch.nn.Module):
    def __init__(self, model_name, model_level, GNN_layers, num_classes, Bias, act_fun, Weight_Initializer, dropout_rate,
                 pred_hidden_dims=[], concat=True, bn=True,
                 add_self=False, args=None):
        self.GNN_layers = GNN_layers
        self.input_dim = GNN_layers[0]
        self.output_dim = GNN_layers[-1]
        self.num_GNN_layers = len(GNN_layers)
        self.num_classes = num_classes


        if model_name == 'GCN+GAP':
            super(Graph_Net, self).__init__()
            print ('Graph_Net Input_Dimension:', self.input_dim)

            print ('Graph_Net Number_of_Layers:', self.num_GNN_layers)

            print ('Graph_Net Output_Dimension:', self.output_dim)



            self.args = args
            if act_fun == 'ReLu':
                self.act_fun = F.relu
                print('ReLu is Selected.')
            elif act_fun == 'eLu':
                self.act_fun = nn.functional.elu
                print('eLu is Selected.')


            self.GConvs = torch.nn.ModuleList()
            #self.gcn1 = GCNConv(self.input_dim, self.hidden_dim, bias=Bias)
            #torch.nn.init.ones_(self.gcn1.lin.weight)
            #self.GConvs.append(self.gcn1)

            for i in range(self.num_GNN_layers):
                if self.num_GNN_layers == 1:
                    self.GConvs.append(GCNConv(self.input_dim, self.output_dim, bias=Bias))
                elif self.num_GNN_layers > 1:
                    if i == 0:
                        self.GConvs.append(GCNConv(self.input_dim, self.GNN_layers[i], bias=Bias))
                    elif i == self.num_GNN_layers-1:
                        self.GConvs.append(GCNConv(self.GNN_layers[i-1], self.output_dim, bias=Bias))
                        print("passed")
                    elif 0 < i < self.num_GNN_layers-1:
                        self.GConvs.append(GCNConv(self.GNN_layers[i-1], self.GNN_layers[i], bias=Bias)) 
                else:
                    print("please enter layer config")

                

            #self.GConvs.append(GCNConv(self.hidden_dim, self.output_dim))
            print('len(self.GConvs):', len(self.GConvs))
            self.dropout = nn.Dropout(p=dropout_rate)

            if model_level == 'node':
                self.readout = IdenticalPool()
            else:
                self.readout = GlobalMeanPool()

            #self.ffn = nn.Linear(self.output_dim, self.output_dim, bias=False)
            self.ffn = nn.Linear(self.output_dim, self.num_classes, bias=Bias)
            torch.nn.init.ones_(self.ffn.weight)

            mean = 0
            std = 0.1
            self.initialize_weights(Weight_Initializer, Bias, mean, std)



        else:
            print('Model is not defined well.')

    def initialize_weights(model, Weight_Initializer, Bias, mean, std):
        # 1. Xavier Normal_.  2. Kaiming Normal_.  3. Uniform (0,0.1std)
        if Weight_Initializer == 1:                                             #.      1. Xavier Normal_.
            for i,layers in enumerate(model.children()):
                #print("no condition: ", i, layers)
                if isinstance(layers, torch.nn.ModuleList):
                    for j, layer in enumerate(layers.modules()):
                        if isinstance(layer, GCNConv):
                            torch.nn.init.xavier_normal_(layer.lin.weight)
                            #print(i, j, "   GCN modules:    ",layer)
                            #print("done")
                            if Bias:
                                layer.bias.data.zero_()
                            #print("layer weights: ", layer.state_dict().get('lin.weight'))
                            #print("layer bias: ", layer.bias)
                        else: 
                            #print(i, j, "   components of a GCN :     ", layer)
                            pass
                if isinstance(layers, torch.nn.Linear):
                    #print("Dense before update: ", layers.weight)
                    torch.nn.init.xavier_normal_(layers.weight)
                    if Bias:
                        layers.bias.data.zero_()
                    #print(i, "   Dense:     ", layers)
                    #print("Dense after update: ", layers.weight)
                    #print("layer bias: ", layers.bias)
                elif isinstance(layers, (GlobalMeanPool)):
                    #print(i, "   GAP:      ",  layers)
                    pass
                elif isinstance(layers, (IdenticalPool)):
                    #print(i, "   IAP:      ",  layers)
                    pass

        if Weight_Initializer == 2:                                             #.      2. Kaiming Normal_.
            for i,layers in enumerate(model.children()):
                #print("no condition: ", i, layers)
                if isinstance(layers, torch.nn.ModuleList):
                    for j, layer in enumerate(layers.modules()):
                        if isinstance(layer, GCNConv):
                            torch.nn.init.kaiming_normal_(layer.lin.weight)
                            #print(i, j, "   GCN modules:    ",layer)
                            #print("done")
                            if Bias:
                                layer.bias.data.zero_()
                            #print(layer.state_dict().get('lin.weight'))
                            #print("layer bias: ", layer.bias)
                        else: 
                            #print(i, j, "   components of a GCN :     ", layer)
                            pass
                if isinstance(layers, torch.nn.Linear):
                    #print("Dense before update: ", layers.weight)
                    torch.nn.init.kaiming_normal_(layers.weight)
                    if Bias:
                        layers.bias.data.zero_()
                    #print(i, "   Dense:     ", layers)
                    #print("Dense after update: ", layers.weight)
                    #print("layer bias: ", layers.bias)
                elif isinstance(layers, (GlobalMeanPool)):
                    #print(i, "   GAP:      ",  layers)
                    pass
                elif isinstance(layers, (IdenticalPool)):
                    #print(i, "   IAP:      ",  layers)
                    pass
                            
        if Weight_Initializer == 3:                                             #.      3. Uniform (0,0.1std)
            for i,layers in enumerate(model.children()):
                #print("no condition: ", i, layers)
                if isinstance(layers, torch.nn.ModuleList):
                    for j, layer in enumerate(layers.modules()):
                        if isinstance(layer, GCNConv):
                            torch.nn.init.normal_(layer.lin.weight.data, mean, std)
                            #print(i, j, "   GCN modules:    ",layer)
                            #print("done")
                            if Bias:
                                layer.bias.data.zero_()
                            #print(layer.state_dict().get('lin.weight'))
                            #print("layer bias: ", layer.bias)
                        else: 
                            #print(i, j, "   components of a GCN :     ", layer)
                            pass
                if isinstance(layers, torch.nn.Linear):
                    #print("Dense before update: ", layers.weight)
                    torch.nn.init.normal_(layers.weight, mean, std)
                    if Bias:
                        layers.bias.data.zero_()
                    #print(i, "   Dense:     ", layers)
                    #print("Dense after update: ", layers.weight)
                    #print("layer bias: ", layers.bias)
                elif isinstance(layers, (GlobalMeanPool)):
                    #print(i, "   GAP:      ",  layers)
                    pass
                elif isinstance(layers, (IdenticalPool)):
                    #print(i, "   IAP:      ",  layers)
                    pass



    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        Output_of_Hidden_Layers = []
        for i in range(self.num_GNN_layers):
            x = self.GConvs[i](x, edge_index)
            x = self.act_fun(x)
            x = self.dropout(x)
            Output_of_Hidden_Layers.append(x)

        pooling_layer_output = self.readout(x, batch)
        ffn_output = self.ffn(pooling_layer_output)
        ffn_output = self.act_fun(ffn_output)

        log_soft = F.log_softmax(ffn_output, dim=1)
        soft = F.softmax(log_soft, dim=1)

        return Output_of_Hidden_Layers, pooling_layer_output, ffn_output, log_soft, soft

GNN_Model = Graph_Net(model_name='GCN+GAP', model_level='graph', GNN_layers=[7, 7], num_classes=2, Bias=True, 
                      act_fun='eLu', Weight_Initializer=1, dropout_rate=0.1)
print('===================================================================================')
print(GNN_Model)
print('===================================================================================')






#dataset = TUDataset(root='data/TUDataset', name='MUTAG')
#Output_of_Hidden_Layers, pooling_layer_output, ffn_output, log_soft, soft = GNN_Model(dataset[0])
#print(Output_of_Hidden_Layers, pooling_layer_output, ffn_output, log_soft, soft)


Graph_Net Input_Dimension: 7
Graph_Net Number_of_Layers: 2
Graph_Net Output_Dimension: 7
eLu is Selected.
passed
len(self.GConvs): 2
Graph_Net(
  (GConvs): ModuleList(
    (0): GCNConv(7, 7)
    (1): GCNConv(7, 7)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (readout): GlobalMeanPool()
  (ffn): Linear(in_features=7, out_features=2, bias=True)
)
[tensor([[ 0.5792, -0.2428,  0.7564, -0.1633, -0.2791, -0.5925, -0.4891],
        [ 0.5792, -0.2428,  0.7564, -0.1633, -0.2791, -0.0000, -0.4891],
        [ 0.5534, -0.0000,  0.7226, -0.1566, -0.2683, -0.5746, -0.0000],
        [ 0.0000, -0.2510,  0.7857, -0.1691, -0.2884, -0.6076, -0.5029],
        [ 0.6241, -0.2592,  0.8149, -0.1749, -0.2975, -0.6222, -0.5164],
        [ 0.5534, -0.2332,  0.7226, -0.1566, -0.2683, -0.5746, -0.4728],
        [ 0.0000, -0.2332,  0.7226, -0.1566, -0.2683, -0.5746, -0.4728],
        [ 0.5534, -0.2332,  0.7226, -0.1566, -0.2683, -0.5746, -0.4728],
        [ 0.6241, -0.2592,  0.8149, -0.1749, -0.2975, -0.6222, -